In [1]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# Load the CSV data
data_file = './Dataset/EURUSD/EURUSD_M30_feature_v.2.csv'
df = pd.read_csv(data_file)
df['DateTime'] = pd.to_datetime(df['DateTime'])


In [3]:
# Load the JSON file
json_file = './Dataset/EURUSD/EURUSD_M30_label_v.2.json'
with open(json_file, 'r') as file:
    label_data = json.load(file)

In [4]:
# Extract time series labels
timeseries_labels = label_data[0]['label']

# Create a new column 'signal' and initialize with NaN
df['signal'] = None

# Map signals to the corresponding time intervals
for label in timeseries_labels:
    start_time = datetime.strptime(label['start'], '%Y-%m-%d %H:%M:%S')
    end_time = datetime.strptime(label['end'], '%Y-%m-%d %H:%M:%S')
    signal = label['timeserieslabels'][0]  # Assuming one label per time interval
        
    # Assign signal to rows within the time range
    mask = (df['DateTime'] >= start_time) & (df['DateTime'] <= end_time)
    df.loc[mask, 'signal'] = signal

# Save the updated DataFrame to a new CSV file
output_file = "./Dataset/EURUSD/EURUSD_M30_features+label_v.2.csv"
df.to_csv(output_file, index=False)

print(f"Updated CSV file saved to {output_file}")


Updated CSV file saved to ./Dataset/EURUSD/EURUSD_M30_features+label_v.2.csv


In [5]:
def convert_to_two(series):
    """
    Converts inner values of consecutive 0s (or 1s) to 2 in a pandas Series.
    The first and last elements of each group of 0s or 1s remain unchanged.
    """
    # Ensure series is numeric and replace non-numeric NaN with np.nan
    series = pd.to_numeric(series, errors='coerce')
    array = series.to_numpy()
    n = len(array)
    result = array.copy()

    i = 0
    while i < n:
        # Ignore NaN values
        if np.isnan(array[i]):
            result[i] = 3
            i += 1
            continue

        # Identify groups of consecutive 0s or 1s
        start = i
        while i < n - 1 and array[i] == array[i + 1] and not np.isnan(array[i]):
            i += 1
        end = i

        # Change inner values to 2
        if end - start > 1:  # At least 3 consecutive values
            result[start + 1:end] = 2

        i += 1

    return pd.Series(result)





# df['signal'] = convert_to_two(df['signal'])

In [6]:
def fill_signal_column(dataframe):
    dataframe['signal'] = dataframe['signal'].fillna(3)
    return dataframe
    # return dataframe

# Apply the function
updated_data = fill_signal_column(df)



In [7]:
# Save the updated dataframe to a new CSV file
updated_file_path = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
updated_data.to_csv(updated_file_path, index=False)